In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [170]:
market_caps = pd.read_csv("data/new/market_caps.csv")
prices = pd.read_csv("data/new/prices.csv")
volumes = pd.read_csv("data/new/volumes.csv")
coins = pd.read_csv("data/new/coins.csv")['coins'].values

In [1]:
#rebalance portfolio of coins given target allocations, rebalance period, and start and end dates
def period_rebalance(coins, ps, period=42, allocations="equal", starting=1000000, from_date="2018-01-01", to_date="2023-01-15"):
    n = len(coins)
    
    if allocations=='equal':
        weights = dict(zip(coins, n*[1./n]))
    else:
        weights = allocations
    
    prices = ps.loc[(ps['date']>=from_date) & (ps['date']<=to_date)]
    prices.reset_index(inplace=True)
    
    dates = prices['date'].to_numpy()
    days = len(dates)
    
    rebalance_df = pd.DataFrame({'date':dates})
    rebalance_df['portfolio_value'] = pd.Series(dtype='float64')
    for coin in coins:
        rebalance_df[coin+'_percent'] = pd.Series(dtype='float64')
        rebalance_df[coin+'_value'] = pd.Series(dtype='float64')
        rebalance_df[coin+'_amount'] = pd.Series(dtype='float64')
        
    rebalance_df.at[0, 'portfolio_value'] = starting
    for coin in coins: 
        rebalance_df.at[0, coin+'_percent'] = 100*weights[coin]
        rebalance_df.at[0, coin+'_value'] = starting*weights[coin]
        rebalance_df.at[0, coin+'_amount'] = starting*weights[coin]/prices.loc[0,coin+'_price']
        
    for i in range(1,days):
        new_total = 0
        for coin in coins:
            coin_amount = rebalance_df.loc[i-1, coin+'_amount']
            new_price = prices.loc[i, coin+'_price']
            new_value = new_price*coin_amount
            rebalance_df.at[i, coin+'_value'] = new_value
            rebalance_df.at[i, coin+'_amount'] = coin_amount
            new_total += new_value
            
        rebalance_df.at[i, 'portfolio_value'] = new_total
        for coin in coins:
            new_coin_percent = rebalance_df.loc[i, coin+'_value']/new_total * 100
            rebalance_df.at[i, coin+'_percent'] = new_coin_percent
            
        
        if i%period == 0:
            for coin in coins:
                new_value = weights[coin]*rebalance_df.loc[i, 'portfolio_value']
                new_amount = new_value/prices.loc[i, coin+'_price']
                new_percent = 100*weights[coin]
                
                rebalance_df.at[i, coin+'_percent'] = new_percent
                rebalance_df.at[i, coin+'_value'] = new_value
                rebalance_df.at[i, coin+'_amount'] = new_amount
        
        
    return rebalance_df

#rebalance top n coins (by current market cap) over a time period when data exists for all coins
def rebalance(n, coins_all, ps, period=42, allocations="equal", starting=1000000):
    coins = coins_all[:n]
    
    not_null = ~prices[coins[0]+"_price"].isna()
    for coin in coins[1:]:
        not_null = not_null & ~prices[coin+"_price"].isna()
        
    dates = ps[not_null]["date"]
    from_date = dates.iloc[0]
    to_date = dates.iloc[-1]
    return period_rebalance(coins, ps, period=period, allocations=allocations, starting=starting, from_date=from_date, to_date=to_date)

In [233]:
df = rebalance(6, coins, prices)

In [234]:
df

,date,portfolio_value,bitcoin_percent,bitcoin_value,bitcoin_amount,ethereum_percent,ethereum_value,ethereum_amount,tether_percent,tether_value,tether_amount,usd-coin_percent,usd-coin_value,usd-coin_amount,binancecoin_percent,binancecoin_value,binancecoin_amount,ripple_percent,ripple_value,ripple_amount
0,2018-10-05,1.000000e+06,16.666667,1.666667e+05,25.334279,16.666667,1.666667e+05,750.089024,16.666667,1.666667e+05,1.666653e+05,16.666667,1.666667e+05,1.656328e+05,16.666667,1.666667e+05,16170.521075,16.666667,1.666667e+05,3.159308e+05
1,2018-10-06,1.007609e+06,16.647634,1.677431e+05,25.334279,16.975568,1.710474e+05,750.089024,16.519769,1.664547e+05,1.666653e+05,16.463345,1.658861e+05,1.656328e+05,17.097094,1.722719e+05,16170.521075,16.296589,1.642059e+05,3.159308e+05
2,2018-10-07,9.912948e+05,16.839665,1.669307e+05,25.334279,17.030417,1.688216e+05,750.089024,16.817369,1.667097e+05,1.666653e+05,16.728391,1.658277e+05,1.656328e+05,17.009712,1.686164e+05,16170.521075,15.574445,1.543887e+05,3.159308e+05
3,2018-10-08,9.918763e+05,16.818828,1.668220e+05,25.334279,17.065015,1.692639e+05,750.089024,16.799379,1.666291e+05,1.666653e+05,16.730764,1.659485e+05,1.656328e+05,17.199423,1.705970e+05,16170.521075,15.386590,1.526159e+05,3.159308e+05
4,2018-10-09,9.991932e+05,16.868732,1.685512e+05,25.334279,17.230290,1.721639e+05,750.089024,16.669066,1.665562e+05,1.666653e+05,16.609528,1.659613e+05,1.656328e+05,17.091892,1.707810e+05,16170.521075,15.530492,1.551796e+05,3.159308e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,2023-01-11,6.405296e+06,16.645277,1.066179e+06,61.144994,17.162746,1.099325e+06,822.837363,16.070351,1.029354e+06,1.029175e+06,16.066554,1.029110e+06,1.028920e+06,17.360074,1.111964e+06,4012.876726,16.694999,1.069364e+06,3.041205e+06
1560,2023-01-12,6.590483e+06,16.697049,1.100416e+06,61.144994,17.351627,1.143556e+06,822.837363,15.631650,1.030201e+06,1.029175e+06,15.625505,1.029796e+06,1.028920e+06,17.381886,1.145550e+06,4012.876726,17.312283,1.140963e+06,3.041205e+06
1561,2023-01-13,6.673921e+06,17.285356,1.153611e+06,61.144994,17.472061,1.166072e+06,822.837363,15.426630,1.029561e+06,1.029175e+06,15.415658,1.028829e+06,1.028920e+06,17.279859,1.153244e+06,4012.876726,17.120437,1.142604e+06,3.041205e+06
1562,2023-01-14,6.829493e+06,17.854034,1.219340e+06,61.144994,17.510711,1.195893e+06,822.837363,15.078625,1.029794e+06,1.029175e+06,15.054001,1.028112e+06,1.028920e+06,17.283343,1.180365e+06,4012.876726,17.219284,1.175990e+06,3.041205e+06
